In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2

image_size = (480, 640)
batch_size =128


train_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/train_data"
test_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/test_data"
valid_dir = "/content/drive/My Drive/Colab Notebooks/new_chess_data/validation_data"


num_classes = 12

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(120, 160),
    batch_size=batch_size,
    class_mode='categorical')


validation_generator = validation_datagen.flow_from_directory(
    valid_dir,
    target_size=(120, 160),
    batch_size=batch_size,
    class_mode='categorical')


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(120, 160),
    batch_size=batch_size,
    class_mode='categorical')


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=l2(0.001),input_shape=(120, 160,3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu',kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(256, (2, 2), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(512, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', kernel_regularizer=l2(0.001)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])


Found 1017 images belonging to 12 classes.
Found 1002 images belonging to 12 classes.
Found 4331 images belonging to 12 classes.


In [3]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 118, 158, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 59, 79, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 57, 77, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 28, 38, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 27, 37, 256)       131328    
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 13, 18, 256)      0

In [5]:
from keras.callbacks import EarlyStopping

earlystop_callback = EarlyStopping(
    monitor='val_accuracy', 
    min_delta=0.001, 
    patience=10, 
    verbose=1, 
    mode='auto',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=100,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size,
    callbacks=[earlystop_callback]
)



Epoch 1/100
7/7 [==============================] - 28s 3s/step - loss: 3.5329 - accuracy: 0.1260 - val_loss: 3.2325 - val_accuracy: 0.1339
Epoch 2/100
7/7 [==============================] - 17s 3s/step - loss: 3.0920 - accuracy: 0.1260 - val_loss: 2.9241 - val_accuracy: 0.1362
Epoch 3/100
7/7 [==============================] - 17s 3s/step - loss: 2.8603 - accuracy: 0.1305 - val_loss: 2.7693 - val_accuracy: 0.1317
Epoch 4/100
7/7 [==============================] - 17s 3s/step - loss: 2.7348 - accuracy: 0.1316 - val_loss: 2.6745 - val_accuracy: 0.1395
Epoch 5/100
7/7 [==============================] - 17s 3s/step - loss: 2.6596 - accuracy: 0.1406 - val_loss: 2.6110 - val_accuracy: 0.1451
Epoch 6/100
7/7 [==============================] - 17s 3s/step - loss: 2.5926 - accuracy: 0.1406 - val_loss: 2.5764 - val_accuracy: 0.1440
Epoch 7/100
7/7 [==============================] - 17s 3s/step - loss: 2.5185 - accuracy: 0.1519 - val_loss: 2.4701 - val_accuracy: 0.1886
Epoch 8/100
7/7 [==========

In [7]:
test_loss, test_acc = model.evaluate(test_generator)

print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

34/34 [==============================] - 878s 27s/step - loss: 0.8845 - accuracy: 0.7502
Test loss: 0.8845298886299133
Test accuracy: 0.7501731514930725


In [1]:
import os
import shutil

parent_dir = "/Users/zuzannaszubartowska/code/crop_colour/data"
train_dir_name = "train"
validation_dir_name = "validation"
split_ratio = 0.25

for folder in os.listdir(parent_dir):
    if os.path.isdir(os.path.join(parent_dir, folder)):
        train_dir = os.path.join(parent_dir, train_dir_name + "_" + folder)
        validation_dir = os.path.join(parent_dir, validation_dir_name + "_" + folder)
        os.makedirs(train_dir, exist_ok=True)
        os.makedirs(validation_dir, exist_ok=True)
        files = os.listdir(os.path.join(parent_dir, folder))
        num_files = len(files)
        num_train_files = int(num_files * split_ratio)
        num_validation_files = int(num_files * split_ratio)
        train_files = files[:num_train_files]
        validation_files = files[num_train_files:num_train_files+num_validation_files]
        for file in train_files:
            src_path = os.path.join(parent_dir, folder, file)
            dst_path = os.path.join(train_dir, file)
            shutil.copy(src_path, dst_path)
        for file in validation_files:
            src_path = os.path.join(parent_dir, folder, file)
            dst_path = os.path.join(validation_dir, file)
            shutil.copy(src_path, dst_path)


In [2]:
import os
from PIL import Image

parent_dir = "/Users/zuzannaszubartowska/code/crop_colour/data/train"

# Loop over each folder in the parent directory
for folder in os.listdir(parent_dir):
    folder_path = os.path.join(parent_dir, folder)
    # Check if the item in the parent directory is a folder
    if os.path.isdir(folder_path):
        print(f"Augmenting images in {folder}...")
        # Loop over each file in the folder
        for file in os.listdir(folder_path):
            file_path = os.path.join(folder_path, file)
            # Check if the item in the folder is a file and if it's an image
            if os.path.isfile(file_path) and file.endswith((".jpg", ".jpeg", ".png")):
                # Open the image using PIL
                image = Image.open(file_path)
                # Flip the image over the vertical axis
                flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)
                # Generate a new filename for the flipped image
                new_file_name = "flipped_" + file
                new_file_path = os.path.join(folder_path, new_file_name)
                # Save the flipped image with the new filename
                flipped_image.save(new_file_path)
        print(f"Done augmenting images in {folder}.")


Augmenting images in train_blackbishop...
Done augmenting images in train_blackbishop.
Augmenting images in train_blackqueen...
Done augmenting images in train_blackqueen.
Augmenting images in train_whiteking...
Done augmenting images in train_whiteking.
Augmenting images in train_blackpawn...
Done augmenting images in train_blackpawn.
Augmenting images in train_blackrook...
Done augmenting images in train_blackrook.
Augmenting images in train_whiteempty...


/var/folders/06/83g1gzl96nj2_cpq0t67k67h0000gn/T/ipykernel_71287/50848984.py:20: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  flipped_image = image.transpose(Image.FLIP_LEFT_RIGHT)


Done augmenting images in train_whiteempty.
Augmenting images in train_blackknight...
Done augmenting images in train_blackknight.
Augmenting images in train_blackking...
Done augmenting images in train_blackking.
Augmenting images in train_whitepawn...
Done augmenting images in train_whitepawn.
Augmenting images in train_whitebishop...
Done augmenting images in train_whitebishop.
Augmenting images in train_whitequeen...
Done augmenting images in train_whitequeen.
Augmenting images in train_whiteknight...
Done augmenting images in train_whiteknight.
Augmenting images in train_blackempty...
Done augmenting images in train_blackempty.
Augmenting images in train_whiterook...
Done augmenting images in train_whiterook.
